# Authorisation

This notebook tests granting and retracting authorisations for different levels of medical actor, and assigning medical staff a permission level.

In [1]:
% Set up required to reset environment
?- cd('~/work'), ['dec'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).

true.
false.
true.

We initially expect that a diagnosis will have been made by a medical staff member authorised to do so. 

The hospital is aware of various labs, which can be certified to perform various tests. Labs can have existing certifications upon being linked to the hospital, or have certifications added or withdrawn.

Staff members have a permission level i.e. nurse or doctor. Different levels are able to perform different actions. A level can have actions granted or revoked, and a staff member can have their permission level changed.

In [2]:
% File: labInitiates.pl

initially(exp_rule(diagnosed(_,_,_,_,Actor), individualAuthorised(diagnosis,Actor))).

%initially(certified(Lab,Test)):- initially(lab(Lab,L)), member(Test,L).
%initiates(certification(Lab,Test,true), certified(Lab,Test),_).
%terminates(certification(Lab,Test,false), certified(Lab,Test),_).
%initiates(labConnection(Lab,L), certified(Lab,Test),_):- member(Test,L).
%initiates(labConnection(Lab,L), lab(Lab,L),_).
%terminates(labWithdrawal(Lab), lab(Lab,_),_).

%initiates(diagnosis(RID,PatID,Test,Result,Actor), diagnosed(RID,PatID,Test,Result,Actor),_).

% New or old staff
initiates(newStaff(Actor,Level), medStaff(Actor,Level),_).
terminates(oldStaff(Actor,Level), medStaff(Actor,Level),_).
terminates(oldStaff(Actor,Level), individualAuthorised(Action,Actor),T):- holdsAt(levelAuthority(Level,Action),T).


% Are staff authorised at time zero?
initially(individualAuthorised(Action,Actor)):- initially(levelAuthority(Level,Action)), initially(medStaff(Actor,Level)).
initiates(newStaff(_,Actor,Level), individualAuthorised(Action,Actor),T):- holdsAt(levelAuthority(Level,Action),T).

% Add or revoke authorities for levels. 
initiates(authorityAdd(Level, Action), levelAuthority(Level,Action),T).
terminates(authorityRemove(Level, Action), levelAuthority(Level, Action),T).
% Add or revoke level authorities to medical staff
initiates(authorityAdd(Level, Action), individualAuthorised(Action, Actor),T):- holdsAt(medStaff(Actor,Level),T).
terminates(authorityRemove(Level,Action), individualAuthorised(Action,Actor),T):- holdsAt(medStaff(Actor,Level),T).

% Demotions or Promotions
terminates(promotion(Actor,NewLevel), medStaff(Actor,OldLevel),T).
initiates(promotion(Actor,NewLevel), medStaff(Actor,NewLevel),T).
initiates(promotion(Actor, NewLevel), individualAuthorised(Action,Actor),T):- holdsAt(levelAuthority(NewLevel,Action),T).
terminates(promotion(Actor, NewLevel), individualAuthorised(Action,Actor),T):- holdsAt(levelAuthority(OldLevel,OldAction),T), \+ holdsAt(authority(NewLevel,OldAction),T).

In [3]:
% File: agents.pl
initially(levelAuthority(nurse,testRequests)).
initially(levelAuthority(doctor, testRequests)).
initially(levelAuthority(doctor, diagnosis)).
initially(levelAuthority(specialist, testRequests)).
initially(levelAuthority(specialist, diagnosis)).
initially(levelAuthority(specialist, brainDeath)).

% TO DO: Need to add in lab test authorisations
%initially(lab('Otago Blood')).
%initially(lab('Wellington Main')).

initially(medStaff('AH',nurse)).
initially(medStaff('EH', nurse)).
initially(medStaff('PS', doctor)).
happensAtNarrative(authorityAdd(acceptor, donorDec),3).
happensAtNarrative(authorityAdd(nurse, nurseAct1),1).
happensAtNarrative(authorityAdd(nurse, nurseAct2),1).
happensAtNarrative(promotion('EH', doctor),2).
happensAtNarrative(authorityRemove(nurse, nurseAct2),4).
happensAtNarrative(newStaff('JW', specialist),0).
happensAtNarrative(oldStaff('JW', specialist),2).
happensAtNarrative(newStaff('GM', acceptor),2).

%happensAtNarrative(certification('WM', 'HIV',true),2).
%happensAtNarrative(labConnection('Auckland Central'),3).
%happensAtNarrative(certification('WM', 'HIV',false),5).

In [4]:
% File: narrative.pl
happensAtNarrative(diagnosis(7,123,'HIV', pos,'JW'),1).
happensAtNarrative(diagnosis(7,123,'HIV', pos,'AH'),1).


In [5]:
?- tick(0).
?- holdsAt(F,0).
?- happensAt(E,0).

true.
F = levelAuthority(nurse, testRequests) ;
F = levelAuthority(doctor, testRequests) ;
F = levelAuthority(doctor, diagnosis) ;
F = levelAuthority(specialist, testRequests) ;
F = levelAuthority(specialist, diagnosis) ;
F = levelAuthority(specialist, brainDeath) ;
F = individualAuthorised(testRequests, AH) ;
F = individualAuthorised(testRequests, EH) ;
F = individualAuthorised(testRequests, PS) ;
F = individualAuthorised(diagnosis, PS) ;
F = medStaff(AH, nurse) ;
F = medStaff(EH, nurse) ;
F = medStaff(PS, doctor) .
E = newStaff(JW, specialist) .

At time one, we know that the history for patient 123 has already been requested, and at this time, the history is granted.

In [6]:
?- tick(1).
?- holdsAt(F,1).
?- happensAt(E,1).

true.
F = levelAuthority(nurse, testRequests) ;
F = levelAuthority(doctor, testRequests) ;
F = levelAuthority(doctor, diagnosis) ;
F = levelAuthority(specialist, testRequests) ;
F = levelAuthority(specialist, diagnosis) ;
F = levelAuthority(specialist, brainDeath) ;
F = individualAuthorised(testRequests, AH) ;
F = individualAuthorised(testRequests, EH) ;
F = individualAuthorised(testRequests, PS) ;
F = individualAuthorised(diagnosis, PS) ;
F = medStaff(AH, nurse) ;
F = medStaff(EH, nurse) ;
F = medStaff(PS, doctor) ;
F = medStaff(JW, specialist) .
E = authorityAdd(nurse, nurseAct1) ;
E = authorityAdd(nurse, nurseAct2) ;
E = diagnosis(7, 123, HIV, pos, JW) ;
E = diagnosis(7, 123, HIV, pos, AH) .

In [7]:
?- happensAt(authorityAdd(Level, Action),1), holdsAt(medStaff(Actor,Level),1).
?- holdsAt(medStaff(Actor,Level),1).
?- initiates(authorityAdd(A,B),C,1).

Level = nurse, Action = nurseAct1, Actor = AH ;
Level = nurse, Action = nurseAct1, Actor = EH ;
Level = nurse, Action = nurseAct2, Actor = AH ;
Level = nurse, Action = nurseAct2, Actor = EH .
Actor = AH, Level = nurse ;
Actor = EH, Level = nurse ;
Actor = PS, Level = doctor ;
Actor = JW, Level = specialist .
A = _2960, B = _2962, C = levelAuthority(_2960, _2962) ;
A = nurse, B = _2962, C = individualAuthorised(_2962, AH) ;
A = nurse, B = _2962, C = individualAuthorised(_2962, EH) ;
A = doctor, B = _2962, C = individualAuthorised(_2962, PS) ;
A = specialist, B = _2962, C = individualAuthorised(_2962, JW) .

At time 2, the earlier history request has been answered.

In [8]:
?- tick(2).
?- holdsAt(individualAuthorised(X,Y),2).
?- holdsAt(F,2).
?- happensAt(O,2).
?- happensAtNarrative(P,2).
?- holdsAt(authority(nurse,X),2).
?- holdsAt(individualAuthorised(Action,Actor),2).

true.
X = testRequests, Y = AH ;
X = testRequests, Y = EH ;
X = testRequests, Y = PS ;
X = diagnosis, Y = PS ;
X = nurseAct1, Y = AH ;
X = nurseAct1, Y = EH ;
X = nurseAct2, Y = AH ;
X = nurseAct2, Y = EH .
F = levelAuthority(nurse, testRequests) ;
F = levelAuthority(doctor, testRequests) ;
F = levelAuthority(doctor, diagnosis) ;
F = levelAuthority(specialist, testRequests) ;
F = levelAuthority(specialist, diagnosis) ;
F = levelAuthority(specialist, brainDeath) ;
F = levelAuthority(nurse, nurseAct1) ;
F = levelAuthority(nurse, nurseAct2) ;
F = individualAuthorised(testRequests, AH) ;
F = individualAuthorised(testRequests, EH) ;
F = individualAuthorised(testRequests, PS) ;
F = individualAuthorised(diagnosis, PS) ;
F = individualAuthorised(nurseAct1, AH) ;
F = individualAuthorised(nurseAct1, EH) ;
F = individualAuthorised(nurseAct2, AH) ;
F = individualAuthorised(nurseAct2, EH) ;
F = medStaff(AH, nurse) ;
F = medStaff(EH, nurse) ;
F = medStaff(PS, doctor) ;
F = medStaff(JW, specialist) .

The expectation of an answer has been fulfilled.

In [9]:
?- tick(3).
?- holdsAt(medStaff(ID,Level),3).
?- holdsAt(authorised(Action,Actor),3).
?- holdsAt(authority(Level, Action),3).
?- holdsAt(X,3).
?- happensAt(D,3).

true.
ID = AH, Level = nurse ;
ID = PS, Level = doctor ;
ID = EH, Level = doctor ;
ID = GM, Level = acceptor .
false.
false.
X = levelAuthority(nurse, testRequests) ;
X = levelAuthority(doctor, testRequests) ;
X = levelAuthority(doctor, diagnosis) ;
X = levelAuthority(specialist, testRequests) ;
X = levelAuthority(specialist, diagnosis) ;
X = levelAuthority(specialist, brainDeath) ;
X = levelAuthority(nurse, nurseAct1) ;
X = levelAuthority(nurse, nurseAct2) ;
X = individualAuthorised(testRequests, AH) ;
X = individualAuthorised(testRequests, PS) ;
X = individualAuthorised(diagnosis, PS) ;
X = individualAuthorised(nurseAct1, AH) ;
X = individualAuthorised(nurseAct2, AH) ;
X = individualAuthorised(testRequests, EH) ;
X = individualAuthorised(diagnosis, EH) ;
X = medStaff(AH, nurse) ;
X = medStaff(PS, doctor) ;
X = medStaff(EH, doctor) ;
X = medStaff(GM, acceptor) .
D = authorityAdd(acceptor, donorDec) .

In [11]:
?- initialiseDEC.
?- run(6).
?- happensAt(B,4).
?- holdsAt(authorised(Action,Actor),4).
?- happensAt(D,5).
?- holdsAt(levelAuthority(Level,Action),5).
?- holdsAt(individualAuthorised(Action,Actor),5).
?- holdsAt(medStaff(X,nurse),4).

true.
true.
B = authorityRemove(nurse, nurseAct2) .
false.
false.
Level = nurse, Action = testRequests ;
Level = doctor, Action = testRequests ;
Level = doctor, Action = diagnosis ;
Level = specialist, Action = testRequests ;
Level = specialist, Action = diagnosis ;
Level = specialist, Action = brainDeath ;
Level = nurse, Action = nurseAct1 ;
Level = acceptor, Action = donorDec .
Action = testRequests, Actor = AH ;
Action = testRequests, Actor = PS ;
Action = diagnosis, Actor = PS ;
Action = nurseAct1, Actor = AH ;
Action = testRequests, Actor = EH ;
Action = diagnosis, Actor = EH ;
Action = donorDec, Actor = GM .
X = AH .

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1527cc64-36a2-4b35-bd8b-8d493ca554fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>